In [150]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords

In [151]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shantanusinghal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [158]:
food_adj = pd.read_csv('food_adj.csv')['adjectives'].tolist()
food_adj

['pleasant',
 'acrid',
 'organic',
 'steamy',
 'crisp',
 'solid',
 'sweet',
 'half',
 'ragout',
 'petite',
 'starchy',
 'stuffed',
 'flavorsome',
 'au jus',
 'fruity',
 'rancid',
 'alcoholic',
 'mashed',
 'blah',
 'roux',
 'extraordinary',
 'fatty',
 'flaky',
 'dull',
 'kosher',
 'pickled',
 'creamery',
 'nuked',
 'toothsome',
 'satin',
 'furry',
 'chocolaty',
 'mixture of',
 'zippy',
 'piping',
 'pureed',
 'fishy',
 'gourmet',
 'sapid',
 'lumpy',
 'nectarous',
 'floury',
 'limp',
 'caked',
 'dipping',
 'sliced',
 'intense',
 'seared',
 'texture',
 'crumbly',
 'boiled',
 'refrigerated',
 'almond flavored',
 'tart',
 'soggy',
 'gamy',
 'mixture',
 'moist',
 'parched',
 'stiff',
 'fried',
 'stringy',
 'sautթ',
 'layered',
 'candied',
 'peppered',
 'smelly',
 'gooey',
 'battered',
 'lively',
 'brittle',
 'butterflied',
 'center cut',
 'tempting',
 'gustatory',
 'frosty',
 'famous',
 'healthy',
 'vicious',
 'simmering',
 'epicurean taste',
 'luscious',
 'soupy',
 'creamed',
 'relish',
 'di

In [179]:
data = pd.read_csv('dataset.csv')
#data.drop(data.columns[-1], axis=1, inplace=True)
data.head(10)

,doc_id,index_in_doc,term
0,9,0,not
1,9,4,sure
2,9,9,what
3,9,14,the
4,9,18,hype
5,9,23,is
6,9,26,but
7,9,30,decided
8,9,38,to
9,9,41,give


In [183]:
def extract_features(data):
    p_doc_id, p_term, adj, wlen, wcount, p_wlen, p_wcount, p_adj, n_wlen, n_wcount, n_adj = None, None, [], [], [], [], [], [], [], [], []
    for index, row in data.iterrows():
        if p_doc_id and p_doc_id == row[0] and p_term in food_adj:
            adj.append(1)
        else:
            adj.append(0)

        # previous features
        p_wlen.append(len(p_term) if p_term else 0)
        p_wcount.append(len(p_term.split(' ')) if p_term else 0)
        p_adj.append(adj[index - 1] if index > 0 else 0)
                
        # update local vars to next row
        p_doc_id = row[0]
        p_term = row['term']
        
        # current features
        wlen.append(len(p_term))
        wcount.append(len(p_term.split(' ')))
        
        # next features
        if index > 0:
            n_wlen.append(len(p_term))
            n_wcount.append(len(p_term.split(' ')))
            n_adj.append(adj[index])
        
    n_wlen.append(0)
    n_wcount.append(0)
    n_adj.append(0)

    return adj, wlen, wcount, p_adj, p_wlen, p_wcount, n_adj, n_wlen, n_wcount

data['has_leading_food_adj'], data['word_length'], data['word_count'], data['pre_has_leading_food_adj'], data['pre_word_length'], data['pre_word_count'], data['nxt_has_leading_food_adj'], data['nxt_word_length'], data['nxt_word_count'] = extract_features(data)

data.head(14)

,doc_id,index_in_doc,term,has_leading_food_adj,word_length,word_count,pre_has_leading_food_adj,pre_word_length,pre_word_count,nxt_has_leading_food_adj,nxt_word_length,nxt_word_count
0,9,0,not,0,3,1,0,0,0,0,4,1
1,9,4,sure,0,4,1,0,3,1,0,4,1
2,9,9,what,0,4,1,0,4,1,0,3,1
3,9,14,the,0,3,1,0,4,1,0,4,1
4,9,18,hype,0,4,1,0,3,1,0,2,1
5,9,23,is,0,2,1,0,4,1,0,3,1
6,9,26,but,0,3,1,0,2,1,0,7,1
7,9,30,decided,0,7,1,0,3,1,0,2,1
8,9,38,to,0,2,1,0,7,1,0,4,1
9,9,41,give,0,4,1,0,2,1,0,4,1


Save to file

In [149]:
data.to_csv('final_data.csv', header=True, index=False)